In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('./data_down/1223_fifo_2wk.csv', sep = '\001')
df.columns = pd.Series(df.columns).str.replace('.+\.', '')
# df = df.iloc[:,6:]
df2 =  pd.read_csv('./data_down/fifo_out_2w.csv', sep = '\001')
df2.columns = pd.Series(df2.columns).str.replace('.+\.', '')
# df = df.iloc[:,6:]
# df = df2

C:\Users\DSCSHA~1\AppData\Local\Temp/ipykernel_4820/1307593119.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df.columns = pd.Series(df.columns).str.replace('.+\.', '')
C:\Users\DSCSHA~1\AppData\Local\Temp/ipykernel_4820/1307593119.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  df2.columns = pd.Series(df2.columns).str.replace('.+\.', '')


In [2]:
def data_(stats):
    fefo = df[df['fifo_fefo'] == stats].sort_values(['sku' , 'received_date' ], ascending=True)
    fefo = fefo[fefo['week_7']!=0]
    fefo['out_amt'] = fefo['week_6'] - fefo['week_7']


    fefo['flag'] = fefo.groupby(
        ['sku', 'ou_code', 'wms_warehouse_id',   'fifo_fefo', 'start_of_week', 'end_of_week']
        )['sku'].transform('count')
    fefo = fefo[fefo['flag'] != 1]

    fefo['flag'] = fefo.groupby(
        ['sku', 'ou_code', 'wms_warehouse_id',   'fifo_fefo', 'start_of_week', 'end_of_week']
        )['out_amt'].transform(min)
    fefo = fefo[fefo['flag'] > 0]
    fefo.tail(8)
    return fefo
def data2_(stats, df):
    fefo = df
    if stats == 'fifo':
        fefo = fefo.sort_values(['sku' , 'received_date'], ascending=True)
    else:
        fefo = fefo.sort_values(['sku' , 'received_date'], ascending=True)
    fefo_out = fefo.groupby(
        ['ou_code', 'wms_warehouse_id', 'fifo_fefo', 'start_of_week', 'end_of_week', 'sku'] , as_index= False
        ).agg(
            old_remain_amt = ('week_7', 'first'), # old remain
            old_remain_location = ('location', 'first'), # 易过期
            old_remain_date = ('received_date', 'first'), 
            old_remain_lock_code = ('lock_codes', 'first'), 

            new_export_amt = ('out_amt', 'mean'),  # new export
            new_export_location = ('location', 'last'),
            new_export_date = ('received_date', 'last'), 
            new_export_lock_codes = ('lock_codes', 'first'), 
    )
    return fefo_out


In [121]:
# fefo_out = fefo_out[~fefo_out['old_remain_location'].str.contains(fefo_out['new_export_location'][0])]

In [3]:
fifo = data_('fifo')
fefo = data_('fefo')
# fifo.head(5).sort_values(['sku' , 'received_date'], ascending=False)

In [5]:
df_out = pd.concat([
    data2_('fefo', fefo), data2_('fifo', fifo)
    ], axis = 0)\
        # .to_csv('./data_up/fifo_fefo_alert_thur.csv', index = None, encoding = 'utf_8_sig')

In [12]:
data2_('fifo', fifo)

,ou_code,wms_warehouse_id,fifo_fefo,start_of_week,end_of_week,sku,old_remain_amt,old_remain_location,old_remain_date,old_remain_lock_code,new_export_amt,new_export_location,new_export_date,new_export_lock_codes
0,HPPXXSHMGS,HPI_SH,fifo,20211216,20211223,6ZD29AA,14.0,"SHIP1,11-04-3,22-09-A",2021-11-30,1001-34,839.250000,"12-07-2,SHIP1,12-12-2",2021-12-07,1001-34
1,HPPXXSHMGS,HPI_SH,fifo,20211216,20211223,C2P04AA,6.0,"SHIP1,16-19-2",2021-11-30,1001-34,3235.500000,"09-03-2,14-08-3,13-09-4",2021-12-16,1001-34
2,HPPXXSHMGS,HPI_SH,fifo,20211216,20211223,CE320A,108.0,P-08-09-2,2021-06-18,1008-34,39.333333,"P-12-23-3,P-02-14-1,P-09-27-3",2021-12-16,1008-34
3,HPPXXSHMGS,HPI_SH,fifo,20211216,20211223,CF226XC,63.0,23-08-3,2021-10-19,1001-34,31.750000,"P-09-01-1,P-06-23-1",2021-10-21,1001-34
4,HPPXXSHMGS,HPI_SH,fifo,20211216,20211223,CF380A,8.5,09-05-3,2021-10-13,1008-34,12.250000,"P-01-05-1,22-10-A",2021-12-16,1008-34
5,HPPXXSHMGS,HPI_SH,fifo,20211216,20211223,CN623AA,18.0,X-21-09-2,2021-06-18,1008-34,25.000000,20-03-3,2021-11-19,1008-34
6,HPPXXSHMGS,HPI_SH,fifo,20211216,20211223,CN624AA,9.0,SMALL-51,2021-08-25,1008-34,29.000000,20-06-3,2021-11-19,1008-34
7,HPPXXSHMGS,HPI_SH,fifo,20211216,20211223,T6N12AA,178.5,"14-04-A,SHIP1",2021-11-30,1001-34,368.250000,14-04-B,2021-12-07,1001-34
8,HPPXXWHWDS,HPI_WH,fifo,20211216,20211223,SU847A,40.0,B-05-53-2,2021-12-02,1001-34,230.000000,C-03-40-2,2021-12-10,1001-34
9,HPPXXWHWDS,HPI_WH,fifo,20211216,20211223,W9044MC,2.0,B-03-47-C,2021-11-17,1002-34,27.500000,A-01-19-2,2021-12-13,1002-34


In [16]:
from datetime import date 
def allsundays(year):
        """
        十年, 
        """
        return pd.Series(pd.date_range(start=str(year), end=str(year+10), 
                            freq="W-FRI").strftime('%Y%m%d'))
fridays = tuple([
        i for i in list(
            allsundays(2021)[allsundays(2021) < date.today().strftime('%Y%m%d')][-int(1):])])[0]

In [17]:
fridays

'20211224'